### Mid-term instructions

This assignment is to help you build several databases for your major research projects. These codes should give you a great amount of ideas how to combine, clean common financial databases for analysis.

We will be looking at six databases: accounting data from annual reports, stock returns data from stock exchanges, analyst reports, CEO characteristics from proxy statements, and board data from proxy statements, firm's patent grants from patent office

To make these databases manageable, I will be using 2007 to 2017, 10 years US data for illustration. For your research project, it is more than enough.

These data are all available via WRDS, feel free to download them if you need more of them. I will point out where to download. For our convinience, I downloaded them already and convert them to a binary format called parquet. It helps pandas to more quickly read in the data and enable us to select columns before read the whole dataset.

In [ ]:
# import libraries
import pandas as pd
import numpy as np


In [ ]:
# you may need to run below code to install some additional libraries, comment it out if you have installed them
!pip install fastparquet pyarrow

In [ ]:
# I can read fundamental_annual data by using pd.read_parquet
funda = pd.read_parquet('https://mfin6210.s3.amazonaws.com/fundamental_annual.pq',
                        columns=['gvkey','cusip','permno','fyear','datadate','at','xrd'])

fundamental data has over 900 columns for an array of accounting variables you can look at the
variable definition: https://wrds-web.wharton.upenn.edu/wrds/ds/compd/funda/index.cfm?navId=83

I manage to only read four columns: permno (firm id), fyear (fiscal year), at(total asset), xrd (R&D expenses)
by issuing columns= parameter.

Of course you can add more variables if you would like. Look at the documentation to see what's in there and grab what you want

In [ ]:
# This dataset is a panel. For illustration purpose, I will use accounting data to serve as the basis
# And merge other datasets to this one. 

next, I read stock return data. It is monthly returns for each firm each month. For variable descriptions:

https://wrds-web.wharton.upenn.edu/wrds/ds/crsp/stock_a/msf.cfm?navId=128

In [ ]:
ret = pd.read_parquet('https://mfin6210.s3.amazonaws.com/stock_return.pq')

In [ ]:
# convert month returns to annual returns as measures of the returns
# and calculate standard deviations of monthly returns for each year as measures of the risk

# HINT: create a new dataframe called std_ret that calculates std of returns by grouping by the 
# dataframe by permno and fyear
# and create a new dataframe called are aret that
# calcualtes annual ret by using product of 1+ret by grouping by permno and date
# finally, combining (merging/joining) returns and risk to a dataframe called stock_return

# Your code:

In [ ]:
# Running next cell will reveal the solution, please work on it first 
# and only peek the solution if you have stucked

In [ ]:
%load 1.py

In [ ]:
# We left merge to accounting data. We keep everything on the left because 
# accounting data is our basis, if you do inner join, we gonna lose more and more observation as we 
# joining more datasets. So for completeness, we left merging datasets to our basis dataset
# and deal with missing values later
df = funda.merge(stock_return,how='left') 
# the common columns to merge on is permno and fyear,
# so I just omit the on= parameter here

Next, we read executive characteristics and compensation dataset

https://wrds-web.wharton.upenn.edu/wrds/ds/comp/execcomp/anncomp/index.cfm?navId=72

In [ ]:
# I only read a few columns for illustration, they are gvkey, a firm identifier, fyear, tdc1 is total compensation, and 
# becameceo is the date CEO took the role.
executive = pd.read_parquet('https://mfin6210.s3.amazonaws.com/exec_chars.pq',
                            columns=['gvkey','fyear','tdc1','ceoann','becameceo'])

In [ ]:
# executive chars data lists top 5 executives in the company, suppose we only need CEO data
# write code to subset the rows where ceoann='CEO', save this subset to dataframe "ceos",
# After keep only CEOs, drop duplicates at gvkey, fyear level to form a firm-year panel
# Your code:


In [ ]:
%load 2.py

In [ ]:
# merge CEO characteristics back to df
df = df.merge(ceos,how='left') # this time, the merging key is gvkey and fyear

Next, we merge director information from ISS (also called riskmetrics)

https://wrds-web.wharton.upenn.edu/wrds/ds/riskmetrics/rmdirectors/index.cfm?navId=245

In [ ]:
# each row represents an individual director for a company (cusip) for each year (need to derive from meetingdate)
directors = pd.read_parquet('https://mfin6210.s3.amazonaws.com/directors.pq',
                            columns=['cusip','meetingdate','director_detail_id','classification'])

In [ ]:
# write code to convert meetingdate to fiscal year, if the month < 7, then it is the calendar year - 1
# if the month >= 7, it is the calendar year
# HINT: first convert meetingdate to pandas' datetime format
# your code

In [ ]:
%load 3.py

In [ ]:
# these data comes from proxy statements, sometime company switch their reporting schedule so we may have duplicated reporting
# in some years, but these cases are rare. For our purpose, we need to make sure certain director only appear once in a year
# Therefore, we drop duplicated directors in each firm-year
# (this is cruel way of dealing with duplicates, but since the impact is small, we just force drop the duplicates)
directors = directors.drop_duplicates(['cusip','fyear','director_detail_id'])

In [ ]:
# We calculate an indicator to indicate the director is an independent director
directors['independence'] = directors['classification'].str.contains('I')*1

Here, we calculate two measures:
1. Board size
2. the fraction of independent directors

In [ ]:
# write code to count unique directors in each firm-year as the measure of board size
# write code to calculate the fraction of independent directors
# Your code


In [ ]:
%load 4.py

In [ ]:
independence = (independence / boardsize).rename('independence') # calculate the fraction

In [ ]:
board = pd.concat([boardsize,independence],axis=1).reset_index()

In [ ]:
df = df.merge(board,how='left')

Next, we will merge the analyst report dataset, which contains analyst's forecast EPS for certain company. We will create a few measures there:

https://wrds-web.wharton.upenn.edu/wrds/ds/ibes/det/index.cfm?navId=223

1. Analyst coverage: Number of analysts are there to predict the company's EPS
1. Analyst forecast volatility: Std of analyst's forecasts, a measure of firm's information opacity
1. Analyst's forecast level: The median forecast EPS from all analyst for that firm-year

In [ ]:
# Again, to make the data managable, I only keep firm id (cusip), forecasting date, actual value of forecast and analyst code
# The whole data file is very big, please only select columns that you need
analyst = pd.read_parquet('https://mfin6210.s3.amazonaws.com/analyst_eps.pq',
                          columns=['cusip','fpedats','value','analys'])

In [ ]:
# write code to calculate coverage, forecast volatility and forecast level, create three dataframes:
# coverage, analyst_volatility, analyst_median
# HINT: group by cusip and fpedats
# Your code:


In [ ]:
%load 5.py

In [ ]:
# The forecast is made for every reporting period

In [ ]:
analyst = pd.concat([coverage,analyst_median,analyst_volatility],axis=1).reset_index()

In [ ]:
# for us to merge the analyst data back to df, we need to do two things:
# write code to:
# 1. the cusip in analyst dataset is only first 8 digits, so we need to convert df's cusip to the first 8 digits from 9 digits
# 2. rename analyst's fpedats to datadate, so we can match on column's name for pandas to merge
# you code:


In [ ]:
%load 6.py

In [ ]:
df = df.merge(analyst,how='left')

Finally, we will merge number of patents granted for each firm-year as a measure of innovation

I have already cleaned the data for us.

In [ ]:
pat_count = pd.read_parquet('https://mfin6210.s3.amazonaws.com/patents.pq')

In [ ]:
df = df.merge(pat_count,how='left')

In [ ]:
# Checkpoint

if len(list(df))==18:
    print('congrants! You have successfully completed the exercise')
else:
    assert len(list(df))==18, 'Sorry, you did not complete the exercise'